In [11]:
import os

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_TOKEN")

# Préparer l’URL HTTPS avec token
repo_url = f"https://{token}@github.com/MAthysGallay/PER.git"

# Cloner le repo dans /kaggle/working
os.system(f"git clone {repo_url} /kaggle/working/PER")


fatal: destination path '/kaggle/working/PER' already exists and is not an empty directory.


32768

In [12]:
!ls


annotations		       mini_coco		thermal_coco
annotations_trainval2017.zip   pipeline_yolov5.ipynb	thermal_yolo
data.yaml		       README.md		yolov8n.pt
download_mini_coco_animals.py  runs
image_thermal_converter.py     thermal_augmentation.py


In [13]:
%cd PER
!python download_mini_coco_animals.py
!python image_thermal_converter.py
!python thermal_augmentation.py

[Errno 2] No such file or directory: 'PER'
/kaggle/working/PER
Extraction annotations...
Nombre d'images trouvées : 25465
Téléchargement images: 100%|███████| 118287/118287 [00:00<00:00, 3299932.40it/s]
Images téléchargées : 0
Mini-COCO (animaux uniquement) généré
Conversion RGB → Thermal: 100%|███████████████| 500/500 [00:05<00:00, 86.75it/s]
Conversion thermique terminée !
Images enregistrées dans thermal_coco/images
Augmentation thermique: 100%|█████████████████| 500/500 [00:08<00:00, 61.74it/s]
Augmentation thermique terminée !


In [14]:
!pip install ultralytics

In [15]:
import torch
import os

print("Torch CUDA available:", torch.cuda.is_available())

Torch CUDA available: True


In [16]:
!git status
!git switch model_yanis
!ls

On branch model_yanis
Your branch is up to date with 'origin/model_yanis'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data.yaml
	runs/
	thermal_yolo/
	yolov8n.pt

nothing added to commit but untracked files present (use "git add" to track)
Already on 'model_yanis'
Your branch is up to date with 'origin/model_yanis'.
annotations		       mini_coco		thermal_coco
annotations_trainval2017.zip   pipeline_yolov5.ipynb	thermal_yolo
data.yaml		       README.md		yolov8n.pt
download_mini_coco_animals.py  runs
image_thermal_converter.py     thermal_augmentation.py


In [17]:
import os
import glob
import random
import shutil

# CHEMINS CORRIGÉS POUR L'ENVIRONNEMENT KAGGLE
SOURCE_DIR = "/kaggle/working/PER/thermal_coco/augmented"
SOURCE_LABELS = "/kaggle/working/PER/mini_coco/labels"
TARGET_DIR = "thermal_yolo"

print(f"Source exists: {os.path.exists(SOURCE_DIR)}")
print(f"Labels exist: {os.path.exists(SOURCE_LABELS)}")
os.makedirs(TARGET_DIR, exist_ok=True)

# YOLO folders
folders = ["images/train", "images/val", "labels/train", "labels/val"]
for f in folders:
    os.makedirs(os.path.join(TARGET_DIR, f), exist_ok=True)

# Récupérer toutes les images
images = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
print(f"Images trouvées : {len(images)}")

if len(images) == 0:
    print("ERREUR : Aucune image trouvée!")
else:
    # Train/val split
    train_ratio = 0.8
    random.shuffle(images)
    split = int(len(images) * train_ratio)
    
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}")
    
    # Copier train
    for img_path in train_imgs:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(TARGET_DIR, "images/train", img_name))
        
        # Récupérer le label original (sans _aug)
        base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
        label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
        label_dst = os.path.join(TARGET_DIR, "labels/train", img_name.replace('.jpg', '.txt'))
        
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
    
    # Copier val
    for img_path in val_imgs:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(TARGET_DIR, "images/val", img_name))
        
        base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
        label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
        label_dst = os.path.join(TARGET_DIR, "labels/val", img_name.replace('.jpg', '.txt'))
        
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
    
    print("Dataset YOLO créé")


Source exists: True
Labels exist: True
Images trouvées : 2500
Train: 2000, Val: 500
Dataset YOLO créé


In [18]:
# Classes animales uniquement (celles utilisées dans download_mini_coco_animals.py)
ANIMAL_CLASSES = {
    14: "bird",
    15: "cat",
    16: "dog",
    17: "horse",
    18: "sheep",
    19: "cow",
    20: "elephant",
    21: "bear",
    22: "zebra",
    23: "giraffe",
}

classes = list(ANIMAL_CLASSES.values())
print(f"Classes animales : {classes}")

yaml_content = f"""
train: {TARGET_DIR}/images/train
val: {TARGET_DIR}/images/val

nc: {len(classes)}
names: {classes}
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("data.yaml généré !")


Classes animales : ['bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe']
data.yaml généré !


In [19]:
import os
import sys

# --- Étape A : Désinstallation agressive ---
# Ceci est nécessaire car 'pip install' seul échoue souvent à remplacer correctement les dépendances.

print("--- 1. DÉSINSTALLATION DES PAQUETS EN CONFLIT ---")

# Désinstaller les paquets qui ont été mis à jour par Ultralytics (NumPy) ou qui en dépendent (Matplotlib).
!pip uninstall numpy matplotlib ultralytics -y

# Désinstaller les dépendances cachées qui pourraient encore être liées à NumPy 2.x
# !pip uninstall scipy pandas -y # Décommentez ceci si l'erreur persiste après le premier essai

print("\n--- 2. INSTALLATION DES VERSIONS COMPATIBLES ---")

# 1. Installer une version sûre de NumPy (NumPy < 2.0.0)
# La version 1.26.4 est souvent la dernière version stable de la série 1.x.
!pip install numpy==1.26.4 
# Si vous rencontrez toujours des problèmes, changez à 1.23.0.

# 2. Réinstaller Ultralytics (il utilisera désormais le NumPy 1.26.4)
!pip install ultralytics 

# 3. Réinstaller Matplotlib (doit être installé après NumPy)
!pip install matplotlib


--- 1. DÉSINSTALLATION DES PAQUETS EN CONFLIT ---
Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Found existing installation: matplotlib 3.10.8
Uninstalling matplotlib-3.10.8:
  Successfully uninstalled matplotlib-3.10.8
Found existing installation: ultralytics 8.3.240
Uninstalling ultralytics-8.3.240:
  Successfully uninstalled ultralytics-8.3.240

--- 2. INSTALLATION DES VERSIONS COMPATIBLES ---
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mne 1.10.2 requires matplotlib>=3.7, which is not installed.
ydata-profiling 4.17.0 requires matplotlib<=3.10,>=3.5, which is not installed.
arviz 0.21.0 requires mat

In [20]:
from ultralytics import YOLO

# 1. Spécifier le nom du modèle et le chemin de sortie
# Utilisation de yolov8n.pt comme poids pré-entraînés (transfert learning)
MODEL_NAME = 'yolov8n.pt' 
RUN_NAME = 'thermal_yolov5_custom' 
IMG_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 100

# 2. Charger le modèle
model = YOLO(MODEL_NAME) 

print(f"Début de l'entraînement du modèle {MODEL_NAME} avec {EPOCHS} epochs.")

data_path = "/kaggle/working/PER/data.yaml"

# 3. Entraîner le modèle
# Le modèle entraîné (best.pt) sera sauvegardé sous runs/detect/RUN_NAME/weights/best.pt
results = model.train(
    data= data_path,       # Fichier de configuration du dataset généré ci-dessus
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    epochs=EPOCHS,
    name=RUN_NAME,          # Nom du répertoire de sortie dans runs/detect
    project='runs/train',
    amp=False# Dossier parent pour l'entraînement
)

# --- Chemin vers le modèle entraîné ---
# Le chemin du fichier 'best.pt' est dynamique et est contenu dans l'objet 'results'
# On peut le reconstruire comme suit :
BEST_WEIGHTS_PATH = os.path.join(results.save_dir, 'weights', 'best.pt')


# 4. Charger les poids 'best.pt' générés
best_model = YOLO(BEST_WEIGHTS_PATH)

# 5. Exporter le modèle entraîné au format ONNX
path_to_onnx = best_model.export(
    format='onnx',
    imgsz=IMG_SIZE
)

print(f"\n--- SUCCESS ---")
print(f"Modèle entraîné sauvegardé dans : {BEST_WEIGHTS_PATH}")
print(f"Modèle ONNX exporté vers : {path_to_onnx}")

Début de l'entraînement du modèle yolov8n.pt avec 100 epochs.
Ultralytics 8.3.240 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/PER/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=thermal_yolov5_custom2, nbs=64, nms=False, opset=None, optimi

error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/data/base.py", line 376, in __getitem__
    return self.transforms(self.get_image_and_label(index))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-pack